##  Arbitrage Pricing Theorem (APT) 

Despite all merits that CAPM have, it is not free from critism some of which are listed above. Stephen Ross 1976 attempted to address these critics and propose `Arbitrage Pricing Theorem` (APT), which is based on the identifying various systematic risk factors, which is rather appealing in that it is not realistic to assume that single systematic risk factor affects stock returns. However, inclusion of sources of macroeconomic variable can provide us a better understanding about the stock returns. So, the main goal of APT is to improve portfolio performance by better understanding portfolio creation and evaluation. 

The Arbitrage Pricing Model was proposed by Stephen Ross and predicts the relationship between the return of the portfolio and the return of a single asset through several independent macroeconomic variables. The APT has following assumptions (Bodie et al.(2008)):

* Security returns can be described by a factor model;
* There are sufficient securities to diversify away idiosyncratic risk; and
* Well-functioning security markets do not allow for the persistence of arbitrage opportunities.

We can generalize the APT model as follows:

\begin{equation}
r_i- r_f = \alpha_i + \beta_{im}(r_m-r_f) +...+ \beta_{ij}(r_j-r_f)  + \epsilon_i
\end{equation}

where
 * $r_i$: Expected return of the $i^{th}$ stock
 * $\alpha_i$: asset’s return coming from technical analysis
 * $r_m$ and $r_j$: common risk factors
 * $\beta_{im}$: systematic factor of $m^{th}$ factor
 * $\epsilon_i$: Error term 

Now, the question is how do we find the proper macroeconomic factors? We are looking for some important risk factor that raise the concern of the investors.

[Chen, Roll and Ross (1986)]('https://www.jstor.org/stable/2352710') address this issue by listing following variables:

* Inflation rate
* Growth rate in industrial production
* Term structure
* Spread between high-grade and low-grade bond
* Real interest rate
* Oil prices

For the sake of practice and to equip yourself with additional information, I suggest you to use another API here. What about using `AlphaVantage`. Alpha Vantage is a data providing company partnered with major exchanges and institutions. Using Alpha Vantage' API, it is possible to access stock prices, stock fundamentals, Forex information and many more with different time interval- e.g. intraday, daily, weekly, an so on. For more information, please see https://www.alphavantage.co/support/#support[Alpha Vantage's website.]

Let me change the tickers so that we can analyze different companies. This time, let's extract the stock prices of two social media companies, which are:
* Twitter
* Facebook

Having listed some relevant variables, we are now able to run the `APT` analysis. We could proceed by retrieving the data first. Again, we refer to the relate APIs to extract these data. Thanks to `Yahoo Finance` and `FRED`, we have stock prices, risk free rate, and stock market index (S&P-500) from the preceding part.

Let's import all the necessary libraries first.

In [3]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from fredapi import Fred
import yfinance as yf
import datetime

In [6]:
def getMonthlyData(symbol):
        parameters = {'function': 'TIME_SERIES_MONTHLY_ADJUSTED', 'symbol': symbol,
                       'outputsize':'full',
                       'datatype': 'csv', 
                       'apikey': 'LL1WA15IW41XV2T2'}

        response = requests.get('https://www.alphavantage.co/query', params=parameters)

        csvText = StringIO(response.text)
        data = pd.read_csv(csvText, index_col='timestamp')
        return data

Before starting, here is a little trick for you. The ticker of the `S&P-500` is `SPY` in Alpha Vantage API.

In [7]:
tickers = ['TWTR', 'META', 'SPY']
df = []
for i in tickers:
    df.append(getMonthlyData(i)[::-1]['close']['2015-01-01':'2021-05-01'])
stocks = pd.DataFrame(df).T
stocks.columns =  tickers

In [8]:
stocks.head()

,TWTR,META,SPY
timestamp,,,
2015-01-30,37.53,75.910,199.45
2015-02-27,48.08,78.970,210.66
2015-03-31,50.08,82.215,206.43
2015-04-30,38.96,78.770,208.46
2015-05-29,36.67,79.190,211.14


As per the article of Chen, Roll and Ross (1986), I want to pick the following three variables:
    
* Growth rate in industrial production

* Inflation rate

* Stock Index

These variables will be your independent variables as they have appeared to explain a significant portion of the time-series variability of stock returns.

At this point, to extract the macroeconomic variable, `FRED API` would be a convenient database:

In [9]:
fred = Fred(api_key='78b14ec6ba46f484b94db43694468bb1')

In [10]:
fred.search('industrial')

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
INDPRO,INDPRO,2022-10-03,2022-10-03,Industrial Production: Total Index,1919-01-01,2022-08-01,Monthly,M,Index 2017=100,Index 2017=100,Seasonally Adjusted,SA,2022-09-15 08:19:02-05:00,79,The industrial production (IP) index measures ...
IPB50001N,IPB50001N,2022-10-03,2022-10-03,Industrial Production: Total Index,1919-01-01,2022-08-01,Monthly,M,Index 2017=100,Index 2017=100,Not Seasonally Adjusted,NSA,2022-09-15 08:19:08-05:00,37,The industrial production (IP) index measures ...
IPB50001SQ,IPB50001SQ,2022-10-03,2022-10-03,Industrial Production: Total Index,1919-01-01,2022-04-01,Quarterly,Q,Index 2017=100,Index 2017=100,Seasonally Adjusted,SA,2022-09-15 08:19:12-05:00,22,The industrial production (IP) index measures ...
IPB50001NQ,IPB50001NQ,2022-10-03,2022-10-03,Industrial Production: Total Index,1919-01-01,2022-04-01,Quarterly,Q,Index 2017=100,Index 2017=100,Not Seasonally Adjusted,NSA,2022-09-15 08:19:16-05:00,11,The industrial production (IP) index measures ...
IPB50001A,IPB50001A,2022-10-03,2022-10-03,Industrial Production: Total Index,1919-01-01,2021-01-01,Annual,A,Index 2017=100,Index 2017=100,Not Seasonally Adjusted,NSA,2022-08-25 21:05:34-05:00,7,The industrial production (IP) index measures ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PRINTO01CLQ657S,PRINTO01CLQ657S,2022-10-03,2022-10-03,Production: Industry: Total Industry: Total In...,1991-07-01,2022-04-01,Quarterly,Q,Growth rate previous period,Growth rate previous period,Seasonally Adjusted,SA,2022-09-14 15:07:59-05:00,1,OECD descriptor ID: PRINTO01 OECD unit ID: GPS...
IPG33312NQ,IPG33312NQ,2022-10-03,2022-10-03,Industrial Production: Manufacturing: Durable ...,1972-01-01,2022-04-01,Quarterly,Q,Index 2017=100,Index 2017=100,Not Seasonally Adjusted,NSA,2022-09-15 08:21:07-05:00,1,The industrial production (IP) index measures ...
IPG33312SQ,IPG33312SQ,2022-10-03,2022-10-03,Industrial Production: Manufacturing: Durable ...,1972-01-01,2022-04-01,Quarterly,Q,Index 2017=100,Index 2017=100,Seasonally Adjusted,SA,2022-09-15 08:21:07-05:00,1,The industrial production (IP) index measures ...


In [11]:
ind = fred.get_series('INDPRO')
ind = ind['2015-01-01':'2021-05-01']

In [12]:
ind.head()

2015-01-01    102.7980
2015-02-01    102.1554
2015-03-01    101.8077
2015-04-01    101.2285
2015-05-01    100.7536
dtype: float64

In [13]:
fred.search('potential inflation')

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
FEDTARMDLR,FEDTARMDLR,2022-10-03,2022-10-03,Longer Run FOMC Summary of Economic Projection...,2012-01-25,2022-09-21,Not Applicable,NA,Percent,%,Not Seasonally Adjusted,NSA,2022-09-21 14:08:02-05:00,53,The longer-run projections are the rates of gr...
GDPC1CTMLR,GDPC1CTMLR,2022-10-03,2022-10-03,Longer Run FOMC Summary of Economic Projection...,2009-02-18,2022-09-21,Not Applicable,NA,Fourth Quarter to Fourth Quarter Percent Change,Fourth Qtr. to Fourth Qtr. % Chg.,Not Seasonally Adjusted,NSA,2022-09-21 14:08:06-05:00,32,The longer-run projections are the rates of gr...
UNRATECTMLR,UNRATECTMLR,2022-10-03,2022-10-03,Longer Run FOMC Summary of Economic Projection...,2009-02-18,2022-09-21,Not Applicable,NA,"Fourth Quarter, Percent","Fourth Qtr., %",Not Seasonally Adjusted,NSA,2022-09-21 14:08:07-05:00,16,The longer-run projections are the rates of gr...
GDPC1MDLR,GDPC1MDLR,2022-10-03,2022-10-03,Longer Run FOMC Summary of Economic Projection...,2015-06-17,2022-09-21,Not Applicable,NA,Fourth Quarter to Fourth Quarter Percent Change,Fourth Qtr. to Fourth Qtr. % Chg.,Not Seasonally Adjusted,NSA,2022-09-21 14:08:08-05:00,16,The longer-run projections are the rates of gr...
UNRATEMDLR,UNRATEMDLR,2022-10-03,2022-10-03,Longer Run FOMC Summary of Economic Projection...,2015-06-17,2022-09-21,Not Applicable,NA,Percent,%,Not Seasonally Adjusted,NSA,2022-09-21 14:08:10-05:00,14,The longer-run projections are the rates of gr...
FEDTARCTLLR,FEDTARCTLLR,2022-10-03,2022-10-03,Longer Run FOMC Summary of Economic Projection...,2015-06-17,2022-09-21,Not Applicable,NA,Percent,%,Not Seasonally Adjusted,NSA,2022-09-21 14:08:07-05:00,12,The longer-run projections are the rates of gr...
PCECTPICTMLR,PCECTPICTMLR,2022-10-03,2022-10-03,Longer Run FOMC Summary of Economic Projection...,2009-02-18,2022-09-21,Not Applicable,NA,Fourth Quarter to Fourth Quarter Percent Change,Fourth Qtr. to Fourth Qtr. % Chg.,Not Seasonally Adjusted,NSA,2022-09-21 14:08:09-05:00,11,The longer-run projections are the rates of gr...
FEDTARCTMLR,FEDTARCTMLR,2022-10-03,2022-10-03,Longer Run FOMC Summary of Economic Projection...,2015-06-17,2022-09-21,Not Applicable,NA,Percent,%,Not Seasonally Adjusted,NSA,2022-09-21 14:08:09-05:00,11,The longer-run projections are the rates of gr...
GDPC1RHLR,GDPC1RHLR,2022-10-03,2022-10-03,Longer Run FOMC Summary of Economic Projection...,2009-02-18,2022-09-21,Not Applicable,NA,Fourth Quarter to Fourth Quarter Percent Change,Fourth Qtr. to Fourth Qtr. % Chg.,Not Seasonally Adjusted,NSA,2022-09-21 14:08:09-05:00,11,The longer-run projections are the rates of gr...


In [14]:
inf = fred.get_series('CPIEALL')
inf = inf['2015-01-01':'2021-05-01']

In [15]:
inf.head()

2015-01-01    253.383
2015-02-01    254.369
2015-03-01    255.612
2015-04-01    256.168
2015-05-01    257.238
dtype: float64

In [19]:
fred.search('risk free').loc['GS1']['title']

'Market Yield on U.S. Treasury Securities at 1-Year Constant Maturity, Quoted on an Investment Basis'

In [16]:
fred.search('risk free')

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
DGS10,DGS10,2022-10-03,2022-10-03,Market Yield on U.S. Treasury Securities at 10...,1962-01-02,2022-09-29,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2022-09-30 15:19:07-05:00,93,For further information regarding treasury con...
GS10,GS10,2022-10-03,2022-10-03,Market Yield on U.S. Treasury Securities at 10...,1953-04-01,2022-08-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2022-09-01 15:21:10-05:00,73,Averages of business days. For further informa...
WGS10YR,WGS10YR,2022-10-03,2022-10-03,Market Yield on U.S. Treasury Securities at 10...,1962-01-05,2022-09-23,"Weekly, Ending Friday",W,Percent,%,Not Seasonally Adjusted,NSA,2022-09-26 15:20:06-05:00,56,Averages of business days. For further informa...
DGS2,DGS2,2022-10-03,2022-10-03,Market Yield on U.S. Treasury Securities at 2-...,1976-06-01,2022-09-29,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2022-09-30 15:19:10-05:00,82,For further information regarding treasury con...
GS2,GS2,2022-10-03,2022-10-03,Market Yield on U.S. Treasury Securities at 2-...,1976-06-01,2022-08-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2022-09-01 15:21:17-05:00,42,Averages of business days. For further informa...
DGS1,DGS1,2022-10-03,2022-10-03,Market Yield on U.S. Treasury Securities at 1-...,1962-01-02,2022-09-29,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2022-09-30 15:20:03-05:00,79,For further information regarding treasury con...
WGS2YR,WGS2YR,2022-10-03,2022-10-03,Market Yield on U.S. Treasury Securities at 2-...,1976-06-04,2022-09-23,"Weekly, Ending Friday",W,Percent,%,Not Seasonally Adjusted,NSA,2022-09-26 15:21:10-05:00,36,Averages of business days. For further informa...
GS1,GS1,2022-10-03,2022-10-03,Market Yield on U.S. Treasury Securities at 1-...,1953-04-01,2022-08-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2022-09-01 15:21:15-05:00,52,Averages of business days. For further informa...
WGS1YR,WGS1YR,2022-10-03,2022-10-03,Market Yield on U.S. Treasury Securities at 1-...,1962-01-05,2022-09-23,"Weekly, Ending Friday",W,Percent,%,Not Seasonally Adjusted,NSA,2022-09-26 15:21:06-05:00,51,Averages of business days. For further informa...


As you are working with monthly data, it does not make sense to use risk free rate (`DGS3MO`) that you used in CAPM as it is daily rate. So, let's pick a monthly version of it, which is `GS1`.Finally, you need to have risk free rate in APT analysis, as well. To do that, you can make use of `FRED` again. 

In [17]:
risk_free = fred.get_series('GS1')
risk_free = risk_free['2015-01-01':'2021-05-01']

In [18]:
risk_free.head()

2015-01-01    0.20
2015-02-01    0.22
2015-03-01    0.25
2015-04-01    0.23
2015-05-01    0.24
dtype: float64

In [20]:
stocks.describe()

,TWTR,META,SPY
count,76.000000,76.000000,76.000000
mean,30.779211,165.898816,265.277105
std,12.348700,57.789566,53.455978
min,14.620000,75.910000,191.590000
25%,19.030000,118.712500,215.362500
50%,30.545000,166.745000,264.760000
75%,36.497500,193.100000,294.580000
max,77.060000,325.080000,417.300000


Well, it seems like we have 76 observations with varying statistics. For instance, aside from market index, mean of `TWTR` and `FB` are the lowest and highest ones, respectively. In parallel to this observation, we expect to have low standard deviation for `TWTR` and high one for `FB`. Yes, this is what we get, the standard deviation of `TWTR` is 14.3. This is also the lowest one among the stocks at our disposal. As for the `FB`, the standard deviation hits 57.7 and implies that it has the most risky asset as standard deviation is a proxy of risk in finance.

In what follows, as suggested, you need to compute inflation rate, which is nothing but the percentage change of it. Here is how you can do that:

In [21]:
inf_rate = inf.pct_change()

In [22]:
risk_free = risk_free / 12

As you can notice, there is a level difference between stock prices, inflation and industrial production index. To deal with the scaling issue, taking logarithm makes sense:

In [23]:
log_ind = np.log(ind)

To be consistent, you need to drop the last two months (April and May) of stock prices and the last month from inflation rate, and industrial production as industrial production index and inflation rate do not have observation at this date.

In [24]:
returns = stocks.pct_change()
returns.dropna(inplace=True)
returns.head()

,TWTR,META,SPY
timestamp,,,
2015-02-27,0.281108,0.040311,0.056205
2015-03-31,0.041597,0.041092,-0.020080
2015-04-30,-0.222045,-0.041902,0.009834
2015-05-29,-0.058778,0.005332,0.012856
2015-06-30,-0.012272,0.083028,-0.024865


In [28]:
print(len(returns))
print(len(log_ind))
print(len(inf_rate))
print(len(risk_free))

75
77
77
77


In [32]:
log_ind = log_ind.iloc[2:]
inf_rate = inf_rate.iloc[2:]
risk_free = risk_free.iloc[2:]

In [33]:
inf_rate.index = returns.index
log_ind.index = returns.index
risk_free.index = returns.index

After completing the data collection and cleaning process and now we should combine all these variables into single data frame along the columns. Pandas' `concat` function does this for us and create a single data frame. But there is one more step to go. The indeces of the data should be matched properly. The following code does this for us:

In [35]:
apt_df = pd.concat([returns, inf_rate, log_ind, risk_free], axis=1)
apt_df.rename(columns={0:'inf_rate', 1: 'log_ind', 2:'risk_free'}, inplace=True)


In [36]:
apt_df.head()

,TWTR,META,SPY,inf_rate,log_ind,risk_free
timestamp,,,,,,
2015-02-27,0.281108,0.040311,0.056205,0.004887,4.623086,0.020833
2015-03-31,0.041597,0.041092,-0.020080,0.002175,4.617380,0.019167
2015-04-30,-0.222045,-0.041902,0.009834,0.004177,4.612678,0.020000
2015-05-29,-0.058778,0.005332,0.012856,0.003744,4.609689,0.023333
2015-06-30,-0.012272,0.083028,-0.024865,0.000383,4.616209,0.025000


In [38]:
apt_df.corr()

,TWTR,META,SPY,inf_rate,log_ind,risk_free
TWTR,1.000000,0.213690,0.269489,0.146413,0.050798,0.053121
META,0.213690,1.000000,0.647020,0.213363,-0.139196,-0.144107
SPY,0.269489,0.647020,1.000000,0.328461,-0.022025,-0.061258
inf_rate,0.146413,0.213363,0.328461,1.000000,0.121576,-0.075389
log_ind,0.050798,-0.139196,-0.022025,0.121576,1.000000,0.671453
risk_free,0.053121,-0.144107,-0.061258,-0.075389,0.671453,1.000000


As expected, stocks and market index are highly correlated and we observe the similar phenemon among macro variables as they show high correlation, too.

In [40]:
apt_df['excess_return_twtr'] = apt_df['TWTR'] - apt_df['risk_free']
apt_df['excess_return_meta'] = apt_df['META'] - apt_df['risk_free']
apt_df['excess_return_spy'] = apt_df['SPY'] - apt_df['risk_free']

## Running APT

As we did before, assigning new variable to dependent and independent variables faciliates the regression analysis. `excess_return_TWTR` is our dependent variable and it is labeled as y by convention. The rest of the variables are independent variables and they are labeled as `X_TWTR`:

In [43]:
y_twtr = apt_df['excess_return_twtr']
X_twtr = apt_df[['excess_return_spy', 'inf_rate', 'log_ind']]
X_twtr = sm.add_constant(X_twtr, prepend=False)
apt_twtr = sm.OLS(y_twtr, X_twtr)
apt_twtr_results = apt_twtr.fit()

In [44]:
apt_twtr_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     excess_return_twtr   R-squared:                       0.224
Model:                            OLS   Adj. R-squared:                  0.191
Method:                 Least Squares   F-statistic:                     6.815
Date:                Mon, 03 Oct 2022   Prob (F-statistic):           0.000421
Time:                        23:06:46   Log-Likelihood:                 39.666
No. Observations:                  75   AIC:                            -71.33
Df Residuals:                      71   BIC:                            -62.06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
excess_return_spy     0.8552      0.258      3.312      0.001       0.340       1.370
inf_rate              4.4701      7.736      0.578      0.565     -10.955      19.896
log_ind               0.0022      0.627      0.003      0.997      -1.248       1.252
const                -0.0249      2.869     -0.009      0.993      -5.746       5.696
==============================================================================
Omnibus:                        3.946   Durbin-Watson:                   2.383
Prob(Omnibus):                  0.139   Jarque-Bera (JB):                3.232
Skew:                           0.359   Prob(JB):                        0.199
Kurtosis:                       3.720   Cond. No.                     2.17e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.17e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The findings suggest that the only statistically significant variable is market index `excess_return_SPY`, which has a coefficient of 0.6935 indicating a high correlation with dependent variable. Inflation rate and industrial production seem to have no explanatory power.

The same procedure is applied to `FB` given below:

In [45]:
y_meta = apt_df['excess_return_meta']
X_meta = apt_df[['excess_return_spy', 'inf_rate', 'log_ind']]
X_meta = sm.add_constant(X_meta, prepend=False)
apt_meta = sm.OLS(y_meta, X_meta)
apt_meta_results = apt_meta.fit()

In [46]:
apt_meta_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     excess_return_meta   R-squared:                       0.735
Model:                            OLS   Adj. R-squared:                  0.724
Method:                 Least Squares   F-statistic:                     65.70
Date:                Mon, 03 Oct 2022   Prob (F-statistic):           1.91e-20
Time:                        23:11:29   Log-Likelihood:                 107.07
No. Observations:                  75   AIC:                            -206.1
Df Residuals:                      71   BIC:                            -196.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
excess_return_spy     1.0798      0.105     10.273      0.000       0.870       1.289
inf_rate              0.6820      3.149      0.217      0.829      -5.598       6.962
log_ind              -0.1743      0.255     -0.683      0.497      -0.683       0.335
const                 0.8192      1.168      0.701      0.485      -1.510       3.148
==============================================================================
Omnibus:                        3.431   Durbin-Watson:                   2.077
Prob(Omnibus):                  0.180   Jarque-Bera (JB):                2.995
Skew:                           0.233   Prob(JB):                        0.224
Kurtosis:                       3.861   Cond. No.                     2.17e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.17e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Similar to our findings in Twitter, we have single statistically significant variable,`excess_return_SPY`. Its estimated coefficient is 1.01 showing even higher correlation with dependent variable. However, inflation rate and industrial production are again not statisticall significant, so it makes no sense to interpret the estimated coefficients of these variables.